Goal: Usind data about strokes, build a predictor of which conditions are most likely to result in a sroke.

In [ ]:
from config import CONFIG
import gradio as gr
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import classes.BMICalculator as bmic
import classes.Encoders as enc
# import classes.Formatter as fm

### EDA

In [ ]:
# EDA
data_loca= CONFIG["DATA_FILEPATH"]
child_bmi_data = CONFIG["CHILD_BMI"]
data = pd.read_csv(data_loca)
child_bmi_df = pd.read_csv(child_bmi_data)
data.head()

In [ ]:
len(data)

In [ ]:
data_df = data.dropna()
len(data_df)

In [ ]:
# round ages to nearest int
data_df["age"] = data_df["age"].round()

In [ ]:
# can't calc bmi for ages below 2
data_df = data_df[data_df["age"] > 1]

In [ ]:
len(data_df)

In [ ]:
# # get histograms of all data to see spread
# import matplotlib.pyplot as plt
# cols = ["gender", "age", "hypertension", "heart_disease","ever_married", "work_type","Residence_type", "avg_glucose_level","bmi","smoking_status","stroke"]
# for col in cols:
#     # counts = data_df[col].value_counts().plot.barh()
#     # fig, ax = plt.subplots()
#     # ax.set_title(f"{col}")
#     # plt.show()

#     categories = data_df[col].value_counts().index
#     counts = data_df[col].value_counts().values
#     plt.barh(categories, counts)
#     plt.title(col)
#     plt.show()


In [ ]:
#  look at correlations
# import numpy as np
# import pandas as pd
# from scipy.stats import chi2_contingency

# def cramers_v(x, y):
#     confusion_matrix = pd.crosstab(x, y)
#     chi2 = chi2_contingency(confusion_matrix)[0]
#     n = confusion_matrix.sum().sum()
#     phi2 = chi2 / n
#     r, k = confusion_matrix.shape
#     phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
#     r_corr = r - ((r - 1) ** 2) / (n - 1)
#     k_corr = k - ((k - 1) ** 2) / (n - 1)
#     return np.sqrt(phi2corr / min((k_corr - 1), (r_corr - 1)))

# def plot_categorical_correlation(features):
#     num_features = features.shape[1]
#     correlation_matrix = np.zeros((num_features, num_features))

#     for i in range(num_features):
#         for j in range(num_features):
#             correlation_matrix[i, j] = cramers_v(features[:, i], features[:, j])

#     # Create a heatmap to visualize the correlation matrix
#     fig, ax = plt.subplots(figsize=(8, 8))
#     cax = ax.matshow(correlation_matrix, cmap='coolwarm', vmin=0, vmax=1)
#     plt.colorbar(cax)

#     # Add labels to the heatmap
#     feature_names = [f"Feature {i+1}" for i in range(num_features)]
#     ax.set_xticks(np.arange(len(feature_names)))
#     ax.set_yticks(np.arange(len(feature_names)))
#     ax.set_xticklabels(feature_names)
#     ax.set_yticklabels(feature_names)
#     ax.xaxis.tick_bottom()

#     # Rotate the x-axis labels for better visibility
#     plt.xticks(rotation=45, ha='right')

#     # Display the correlation values in each cell
#     for i in range(num_features):
#         for j in range(num_features):
#             ax.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha='center', va='center', color='black')

#     plt.title("Categorical Correlation Matrix (Cramer's V)")
#     plt.xlabel("Features")
#     plt.ylabel("Features")
#     plt.show()


# plot_categorical_correlation(data_df)


In [ ]:

# def plot_correlation_matrix(features):
#     # Calculate the correlation matrix using numpy.corrcoef
#     correlation_matrix = np.corrcoef(features, rowvar=False)

#     # Create a heatmap to visualize the correlation matrix
#     fig, ax = plt.subplots(figsize=(8, 8))
#     cax = ax.matshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
#     plt.colorbar(cax)

#     # Add labels to the heatmap
#     feature_names = [f"Feature {i+1}" for i in range(features.shape[1])]
#     ax.set_xticks(np.arange(len(feature_names)))
#     ax.set_yticks(np.arange(len(feature_names)))
#     ax.set_xticklabels(feature_names)
#     ax.set_yticklabels(feature_names)
#     ax.xaxis.tick_bottom()

#     # Rotate the x-axis labels for better visibility
#     plt.xticks(rotation=45, ha='right')

#     # Display the correlation values in each cell
#     for i in range(len(feature_names)):
#         for j in range(len(feature_names)):
#             ax.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha='center', va='center', color='black')

#     plt.title("Pearson's Correlation Matrix")
#     plt.xlabel("Features")
#     plt.ylabel("Features")
#     plt.show()

# plot_correlation_matrix(data_df)


### BMI conversion data

These files contain the L, M, and S parameters needed to generate exact percentiles and z-scores along with the percentile values for the 3rd, 5th, 10th, 25th, 50th, 75th, 90th, 95th, and 97th percentiles by sex (1=male; 2=female) and single month of age. The smoothed 85th percentile values are included in the BMI-for-age and weight-for-stature tables. Age is listed at the half month point for the entire month; for example, 1.5 months represents 1.0-1.99 months or 1.0 month up to but not including 2.0 months of age. The only exception is birth, which represents the point at birth. To obtain L, M, and S values at finer age or length/stature intervals interpolation could be used.

The LMS parameters are the median (M), the generalized coefficient of variation (S), and the power in the Box-Cox transformation (L). To obtain the value (X) of a given physical measurement at a particular z-score or percentile, use the following equation:

X = M (1 + LSZ)**(1/L), L ≠ 0

Or

X = M exp(SZ), L = 0

where the L, M, and S are the values from the appropriate table corresponding to the age in months of the child (** indicates an exponent, such that M(1+LSZ)**(1/L) means raising (1+LSZ) to the (1/L)th power and then multiplying the M; exp(X) is the exponentiation function, e to the power X). Z is the z-score that corresponds to the percentile. z-scores correspond exactly to percentiles, e.g., z-scores of -1.881, -1.645, -1.282, -0.674, 0, 0.674, 1.036, 1.282, 1.645, and 1.881 correspond to the 3rd, 5th, 10th, 25th, 50th, 75th, 85th, 90th, 95th, and 97th percentiles, respectively.

For example, to obtain the 5th percentile of weight-for-age for a 9-month-old male, we would look up the L, M and S values from the WTAGEINF table, which are L=-0.1600954, M=9.476500305, and S=0.11218624. For the 5th percentile, we would use Z=-1.645. Using the equation above, we calculate that the 5th percentile is 7.90 kg.

To obtain the z-score (Z) and corresponding percentile for a given measurement (X), use the following equation:

((X/M)**L) – 1
Z = ————————-, L≠0
LS

or

Z = ln(X/M)/S ,L=0

where X is the physical measurement (e.g. weight, length, head circumference, stature or calculated BMI value) and L, M and S are the values from the appropriate table corresponding to the age in months of the child (or length/stature). (X/M)**L means raising the quantity (X/M) to the Lth power.

For example, to obtain the weight-for-age z-score of a 9-month-old male who weighs 9.7 kg, we would look up the L, M and S values from the WTAGEINF table, which are L=-0.1600954, M=9.476500305, and S=0.11218624. Using the equation above, we calculate that the z-score for this child is 0.207. This z-score corresponds to the 58th percentile.

Z-scores and corresponding percentiles can be obtained from standard normal distribution tables found in statistics text books. Standard normal tables can also be found on the internet by doing a search on a “standard normal table.” In addition, many computer programs have pre-existing functions that convert Z-scores to percentiles and vice versa.



In [ ]:
# chile BMI conversion df
child_bmi_df.head()

In [ ]:
child_bmi_df.describe()

In [ ]:
# # drop secondary heading (from joint male and female bmi chart)
child_bmi_df = child_bmi_df.drop(index = 219)

In [ ]:
child_bmi_df["Sex"].unique()

In [ ]:
child_bmi_df.dtypes

In [ ]:
child_bmi_df = child_bmi_df.astype(float)
child_bmi_df["Sex"] = child_bmi_df["Sex"].astype(int)

In [ ]:
child_bmi_df.dtypes

In [ ]:
child_bmi_df

In [ ]:
# child_bmi_df

t_age = 168.5
t_gender = 2
t_age_mask = child_bmi_df["Agemos"] == t_age
t_gender_mask = child_bmi_df["Sex"] == t_gender
            
t_lookup_row = child_bmi_df[t_age_mask & t_gender_mask]
t_lookup_row

## Encode data for training

In [ ]:
# split data by age 
child_df = data_df[data_df["age"] <= 20]
adult_df = data_df[data_df["age"] > 20]

In [ ]:
# encode bmi for each df
enc_child_bmi =  enc.ChildBMIEncoder(bmi_table= child_bmi_df, data_table= child_df).encode()
enc_adult_bmi = enc.AdultBMIEncoder(adult_df).encode()

In [ ]:
# def combine_dfs(df1,df2):
#     if len(enc_child_bmi.columns) == len(enc_adult_bmi.columns):
#         new_df = pd.concat([enc_child_bmi,enc_adult_bmi])

#     else:
#         print("Columns must match to concatenate dataframes")
#         return new_df

In [ ]:
if len(enc_child_bmi.columns) == len(enc_adult_bmi.columns):
    new_df = pd.concat([enc_child_bmi,enc_adult_bmi])

else:
    print("Columns must match to concatenate dataframes")

new_df

In [ ]:
age_enc = enc.AgeEncoder(new_df,"age")
new_df = age_enc.encode()
# new_df = encode_age.df()
new_df
# encode_age


In [ ]:
de.encode_bmi(new_df,"bmi_buckets")

new_df

In [ ]:
gend_enc = enc.GenderEncoder(new_df,"gender").encode()
marr_enc = enc.MarriageEncoder(new_df,"ever_married").encode()
res_enc = enc.ResidenceEncoder(new_df,"Residence_type").encode()
smoke_enc = enc.SmokingStatusEncoder(new_df, "smoking_status").encode()

new_df = smoke_enc
new_df

In [ ]:
    # de.encode_gender(new_df)
    # de.encode_marriage(new_df)
    # de.encode_residence(new_df)
    # de.encode_smoke_stat(new_df,"smoking_status")
    # new_df

In [ ]:
new_df["stroke"]

In [ ]:
def get_train_df(df, drop_cols, label_col):
    """
    Input: 
        df with encoded training data
        [drop_cols]; columns names to drop from df
        "label_col"; name of label column
    Output: 
        training data df, label df
    """
    labels = df[label_col]
    df = df.drop(label_col, axis = 1)
    train_df = df.drop(drop_cols, axis = 1)

    return train_df, labels

In [ ]:
# get train df, remove unecceasry columns
# removed avg glucose level for now (for demo)
# drop_cols = ["id", "gender", "age", "ever_married", "work_type", "Residence_type", "bmi","smoking_status", "bmi_buckets","avg_glucose_level"]
drop_cols = ["id", "gender", "ever_married", "work_type", "Residence_type", "bmi","smoking_status", "bmi_buckets","avg_glucose_level", "a0", "a10", "a20", "a30", "a40", "a50", "a60", "a70", "a80"]
train_df, labels = get_train_df(new_df, drop_cols, "stroke")
train_df

In [ ]:
train_df.columns

In [ ]:
labels

In [ ]:
# split into testing and training data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df, labels, test_size = 0.2, random_state = 42)

In [ ]:
X_train.dtypes

In [ ]:
# use SVM for classification
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(X_train, y_train)

In [ ]:
predictions = classifier.predict(X_test)

In [ ]:
classifier.score(X_test,y_test)

In [ ]:
classifier.get_params()

In [ ]:
classifier.predict_proba(X_test)

In [ ]:
# more metrics
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

In [ ]:

classification_report(y_test,predictions)

Try to rerain using k-folds

In [ ]:
# new test-train splits
X_train2, X_test2, y_train2, y_test2 = train_test_split(train_df, labels, test_size = 0.3, random_state = 12)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

#define cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

model = classifier

#use k-fold CV to evaluate model
scores = cross_val_score(model, X_train2, y_train2, scoring=make_scorer(accuracy_score),
                         cv=cv, n_jobs=-1)

scores

In [ ]:
c2 = classifier
c2.fit(X_train2,y_train2)
pred2 = c2.predict(X_test2)
c2.score(X_test2,y_test2)



## DEMO

In [ ]:
# demo support functions

def request_age(age):
    return age

def encode_age(df):
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    categories = pd.cut(df.Age, bins, labels=False)
    df.Age = categories
    return df

def request_sex(sex):
    return "Male" if sex.lower() == "male" else "Female"

def hypertension(text):
    feature = 1 if text.lower() == "yes" else 0
    return feature

def heart_disease(text):
    feature = 1 if text.lower() == "yes" else 0
    return feature

    

def bmi_standard(weight_lb,height_in, age):
    """ 
    Inputs:
        height_in = (height feet, height in)
        weight_lb = 
    """
    height = (height_in[0] * 12) + height_in[1]
    bmi = weight_lb / (height **2)

    return bmi

    

def bmi_imperial(weight_kg, height_m,age):

    bmi = weight_kg / (height_m **2)

    return bmi

def req_marriage(text):
    if text.lower() == "yes":
        feature = "Yes"
    else: 
        feature = "No"
    return feature

def red_residence(text):
    if text.lower()== 

In [ ]:
def bmi_standard(weight_lb,height_in, age):
    """ 
    Inputs:
        height_in = (height feet, height in)
        weight_lb = 
    """
    height = (height_in[0] * 12) + height_in[1]
    bmi = weight_lb / (height **2)

    return bmi

    

def bmi_imperial(weight_kg, height_m,age):

    bmi = weight_kg / (height_m **2)

    return bmi

In [ ]:
# dropdown data
gr.CheckboxGroup(["Male", "Female"], label="Gender?")
gr.Dropdown(label = "Residence Type",choices =["Urban", "Rural"],show_label = True, interactive = True)

gr.Dropdown(label = "Smoking Status",choices = ["smokes","never smoked", "formerly smoked", "Unknown"],show_label = True)

gr.CheckboxGroup(["Urban", "Rural"], label="Residence Type")
# convert to 1 or 0 
gr.CheckboxGroup(["Yes", "No"], label="Hypertension?")
gr.CheckboxGroup(["Yes", "No"], label="Heart Disease?")
gr.Textbox("Age")

In [ ]:
# get units of height and weight
# get standard height as (feet,inches)

'age', 'hypertension', 'heart_disease', 'stroke', 'bmi_under',
       'bmi_healthy', 'bmi_over', 'bmi_obese', 'is_male', 'c_ever_married',
       'live_urban', 'ss_1', 'ss_2', 'ss_3', 'ss_4'
 

In [ ]:
# make an encoder interface (assembly line)

In [ ]:
# make enocde classes (for each function)


In [ ]:
# create a predict dict class and initialize with the object

In [ ]:
# example:


prediction_dict = {
                    "height": None, 
                    "height_unit": None, 
                    "weight": None, 
                    "weight_unit": None, 
                    "gender": None, 
                    "age": None, 
                    "marriage_stat": None, 
                    "residence": None, 
                    "smoke_stat": None, 
                    "hypertension": None,
                    "heart_disease": None
                }

def predict_survival(prediction_dict):
    """ 
    Input: 
        {prediction_dict};
            "height": None, 
            "height_unit": None, 
            "weight": None, 
            "weight_unit": None, 
            "gender": None, 
            "age": None, 
            "marriage_stat": None, 
            "residence": None, 
            "smoke_stat": None, 
            "hypertension": None,
            "heart_disease": None
    Output:
        Preddicton of stroke (Yes or No)
    """

  prediction_dict["height"] = 
        ["height_unit"]
        ["weight"]
        "weight_unit"
        "gender"
        "age"
        "marriage_stat"
        "residence"
        "smoke_stat"
        "hypertension"
        "heart_disease"
    if height_unit == "kg" and weight_unit == "m":
        bmi = bmi_imperial(weight_kg= weight, height_m= height, age= age)
    else:
        bmi = bmi_standard(weight_lb= weight, height_in=height, age = age)
    
    # initialize df
    df = pd.DataFrame.from_dict(
        {
            "age": 0,
            "gender": gender
            "hypertension": 0,
            "heart_disease": 0,
            "bmi" = bmi
            "live_urban": 0,
            "ever_married": marriage_stat,
            "Residence_type": residence,
            "smoking_status": smoke_stat
                    }
    )
    df = de.get_bmi_bucket(data_table = data_df, bmi_table =child_bmi_df )
    df = de.encode_bmi(df,"bmi_buckets")
    # df = de.encode_ages(df,"age")
    df = de.encode_gender (df)
    df = de.encode_marriage(df)
    df = de.encode_residence(df)
    df = de.encode_smoke_stat(df,"smoking_status")

    drop_cols = ["gender", "bmi"]

    pred = classifier.predict(df)
    return {"Stroke likely": float(pred[0]), "Survives": float(pred[1])}


demo = gr.Interface(
    predict_survival,
    [
        gr.Dropdown(["first", "second", "third"], type="index"),
        "checkbox",
        gr.Slider(0, 80, value=25),
        gr.CheckboxGroup(["Sibling", "Child"], label="Travelling with (select all)"),
        gr.Number(value=20),
        gr.Radio(["S", "C", "Q"], type="index"),
    ],
    "label",
    examples=[
        ["first", True, 30, [], 50, "S"],
        ["second", False, 40, ["Sibling", "Child"], 10, "Q"],
        ["third", True, 30, ["Child"], 20, "S"],
    ],
    interpretation="default",
    live=True,
)

if __name__ == "__main__":
    demo.launch()

In [ ]:
import gradio as gr
from classes.BMICalculator import BMICalculator


def greet(name):
    return "Hello " + name + "!"



# def predict_stroke(data):
#     """ 
#     hypertension (y/n)
# heart disease (y/n)
# avg glucose level (?)
# bmi (sub for weight and calculate?)
# ever married (y/n/child)
# age 
# neighborhood (urban, rural)
# gender
# smoking status (former, never, yes,  )
# """
#     if passenger_class is None or embark_point is None:
#         return None
#     df = pd.DataFrame.from_dict(
        



#     )
    
# demo = gr.Interface(fn=greet, inputs="text", outputs="text")
def get_data(data_dict):
    data_dict[age_in]


    # make into df
    df = pd.DataFrame.from_dict(data_dict)

    # make fcn to encode data
    return pred

im = gr.PlayableVideo(value= "" autoplay = True)

def output_video():
    im = gr.PlayableVideo(value= "")

data = {}
age_in = gr.Textbox(label = "Age"),
gend_in = gr.CheckboxGroup(["Male", "Female"], label="Gender?")
res_in = gr.CheckboxGroup(["Urban", "Rural"], label="Residence Type")
hyp_in = gr.CheckboxGroup(["Yes", "No"], label="Hypertension?")
hdis_in = gr.CheckboxGroup(["Yes", "No"], label="Heart Disease?")
smoke_in = gr.CheckboxGroup(label = "Smoking Status",choices = ["smokes","never smoked", "formerly smoked", "Unknown"],show_label = True)
req_bmi = BMICalculator(data_dict)
# gr.Dropdown(label = "Residence Type",choices =["Urban", "Rural"],show_label = True, interactive = True),
# gr.Dropdown(label = "Smoking Status",choices = ["smokes","never smoked", "formerly smoked", "Unknown"],show_label = True)



demo = gr.Interface(fn = get_data, inputs = data,
                    
                    
                     outputs = im, live = True)

demo.launch(share=True)

In [ ]:
# import gradio as gr

# Define the first component that takes some input and produces some output
def function1(input_text):
    output_text = f"Function 1: You entered '{input_text}'"
    return output_text

input_component = gr.inputs.Textbox()
output_component1 = gr.outputs.Textbox()

gr.Interface(fn=function1, inputs=input_component, outputs=output_component1, title="Function 1").launch()

# Define the second component that takes the output of the first component as input
def function2(input_text):
    modified_text = f"Function 2: Modified text from Function 1: '{input_text.upper()}'"
    return modified_text

output_component2 = gr.outputs.Textbox()

gr.Interface(fn=function2, inputs=output_component1, outputs=output_component2, title="Function 2").launch()